In [ ]:
import torch
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import random
import string
import re
from nltk.corpus import stopwords
# nltk.download('stopwords')
import tensorflow as tf

In [ ]:
from sklearn.metrics import classification_report, precision_recall_fscore_support, accuracy_score
from sklearn.metrics import confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout
from tensorflow.keras.optimizers import RMSprop
import numpy as np
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import torch
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/Mobile Prioritazitation/Final_dataset.csv')

In [ ]:
# Assuming df is your DataFrame and 'column1' and 'column2' are the column names you want to join
df['review'] = df['title'].fillna('') + ' ' + df['body'].fillna('')

# Calculating Word2Vec Embeddings

In [ ]:
!pip install gensim

In [ ]:
df.info()

In [ ]:
import gensim

# Tokenize text_list into individual words
tokenized_list = [text.split('_') for text in df['review']]

# Train Word2Vec model on tokenized_list
model = gensim.models.Word2Vec(tokenized_list, min_count=1)#, size = 100

# Calculate Word2Vec embeddings for each tokenized text in text_list
embeddings = []
for tokens in tokenized_list:
  # Average the Word2Vec embeddings of all words in a text
  text_embedding = np.mean([model.wv[word] for word in tokens if word in model.wv], axis=0)
  embeddings.append(text_embedding)

# Convert embeddings list to a PyTorch tensor
embeddings_tensor = torch.tensor(embeddings)


In [ ]:
embeddings_tensor

In [ ]:
import pickle
with open("/content/drive/MyDrive/Mobile Prioritazitation/Word2vec.pickle", "wb") as scores:
    pickle.dump(embeddings_tensor, scores)

In [ ]:
import pickle

with open("/content/drive/MyDrive/Mobile Prioritazitation/Word2vec.pickle", "rb") as scores:
   embeddings = pickle.load(scores)

In [ ]:
embeddings

In [ ]:
embeddings.shape

# CNN with Word2Vec

In [ ]:
import numpy as np
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D,GlobalMaxPooling1D, MaxPooling1D, Flatten, Dense, Dropout


In [ ]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(embeddings, df['sentiment'].values, test_size=0.20, random_state=42)
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

In [ ]:
import numpy as np

# Assuming X_train and X_test are lists, convert them to numpy arrays
X_train = np.array(X_train)
X_test = np.array(X_test)

# Reshape the input data for CNN
X_train_cnn = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test_cnn = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))


# Define the CNN model
cnn_model = Sequential()
cnn_model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(100, 1)))
cnn_model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
cnn_model.add(MaxPooling1D(pool_size=2))
cnn_model.add(Flatten())
cnn_model.add(Dense(units=64, activation='relu'))

cnn_model.add(Dropout(rate=0.2))
cnn_model.add(Dense(units=1, activation='sigmoid'))

# Compile the model
cnn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model
cnn_model.fit(X_train_cnn, y_train, epochs=10, batch_size=32, validation_split=0.1)

# Predict on test data
y_pred_prob = cnn_model.predict(X_test_cnn)
y_pred = (y_pred_prob > 0.5).astype('int')

# # Print classification report
# target_names = ['0', '1']
# print(classification_report(y_test, y_pred, target_names=target_names))

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report


# Print classification report for ensemble model
target_names = ['1 Star', '2 Star', '3 Star','4 Star','5 Star']
print(classification_report(y_test, y_pred, target_names=target_names))

cm = confusion_matrix(y_test, y_pred)
print("Confusion matrix:\n", cm)
# corr, linewidths=.3, cmap="RdBu", annot=True, fmt=""
# Plot confusion matrix
sns.heatmap(cm, annot=True, fmt='g', linewidths=.2, cmap='RdBu')
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.show()
# Calculate and print accuracy, precision, recall, and f1-score
accuracy = accuracy_score(y_test, y_pred)
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred, average='binary')
print("Accuracy: {:.4f}, Precision: {:.4f}, Recall: {:.4f}, F1-score: {:.4f}".format(accuracy, precision, recall, f1_score))


In [ ]:
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
# Macro metrics
precision_macro = precision_score(y_test, y_pred, average='macro')
recall_macro = recall_score(y_test, y_pred, average='macro')
f1_macro = f1_score(y_test, y_pred, average='macro')

# Micro metrics
precision_micro = precision_score(y_test, y_pred, average='micro')
recall_micro = recall_score(y_test, y_pred, average='micro')
f1_micro = f1_score(y_test, y_pred, average='micro')

# Print metrics
print(f'Accuracy: {accuracy}')
print('Macro Metrics:')
print(f'Precision: {precision_macro}')
print(f'Recall: {recall_macro}')
print(f'F1-Score: {f1_macro}')

print('Micro Metrics:')
print(f'Precision: {precision_micro}')
print(f'Recall: {recall_micro}')
print(f'F1-Score: {f1_micro}')

In [ ]:
# Assuming y_test contains the true labels and y_pred contains the predicted labels

# Calculate classification report
target_names = ['1', '2', '3', '4', '5']
report = classification_report(y_test, y_pred, target_names=target_names, output_dict=True)

# Calculate accuracy separately
accuracy = accuracy_score(y_test, y_pred)

# Print precision, recall, f1-score, and accuracy for each class
for label, metrics in report.items():
    if label != 'accuracy':
        class_accuracy = metrics["support"] * metrics["recall"]
        print(f'Class: {label}')
        print(f'Class Accuracy: {class_accuracy}')
        print(f'Precision: {metrics["precision"]}')
        print(f'Recall: {metrics["recall"]}')
        print(f'F1-score: {metrics["f1-score"]}')
        print(f'Support: {metrics["support"]}')


# Print overall accuracy
print(f'Accuracy: {accuracy}')


# CNN 

In [ ]:
import gensim
from gensim.models import FastText
import numpy as np

# Tokenize text_list into individual words
tokenized_list = [text.split('_') for text in df['review']]

# Train FastText model on tokenized_list
model = FastText(tokenized_list, min_count=1)  #, size=100 You can specify the size of the embeddings, here I used size=100 as an example

# Calculate FastText embeddings for each tokenized text in text_list
embeddings = []
for tokens in tokenized_list:
    # Average the FastText embeddings of all words in a text
    text_embedding = np.mean([model.wv[word] for word in tokens if word in model.wv], axis=0)
    embeddings.append(text_embedding)

# Convert embeddings list to a NumPy array
embeddings_array = np.array(embeddings)

# Now you can use embeddings_array as your FastText word embeddings
print(embeddings_array)


In [ ]:
import pickle
with open("/content/drive/MyDrive/Mobile Prioritazitation/FastText.pickle", "wb") as scores:
    pickle.dump(embeddings_array, scores)

In [ ]:
import pickle

with open("/content/drive/MyDrive/Mobile Prioritazitation/FastText.pickle", "rb") as scores:
   embeddings_array = pickle.load(scores)

In [ ]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(embeddings_array, df['sentiment'].values, test_size=0.20, random_state=42)
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

In [ ]:

# Reshape the input data for CNN
X_train_cnn = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test_cnn = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Define the CNN model
cnn_model = Sequential()
cnn_model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(100, 1)))
cnn_model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
cnn_model.add(MaxPooling1D(pool_size=2))
cnn_model.add(Flatten())
cnn_model.add(Dense(units=64, activation='relu'))

cnn_model.add(Dropout(rate=0.2))
cnn_model.add(Dense(units=1, activation='sigmoid'))

# Compile the model
cnn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model
cnn_model.fit(X_train_cnn, y_train, epochs=10, batch_size=32, validation_split=0.1)

# Predict on test data
y_pred_prob = cnn_model.predict(X_test_cnn)
y_pred = (y_pred_prob > 0.5).astype('int')

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report


# Print classification report for ensemble model
target_names = ['1 Star', '2 Star', '3 Star','4 Star','5 Star']
print(classification_report(y_test, y_pred, target_names=target_names))

cm = confusion_matrix(y_test, y_pred)
print("Confusion matrix:\n", cm)
# corr, linewidths=.3, cmap="RdBu", annot=True, fmt=""
# Plot confusion matrix
sns.heatmap(cm, annot=True, fmt='g', linewidths=.2, cmap='RdBu')
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.show()

# Calculate and print accuracy, precision, recall, and f1-score
accuracy = accuracy_score(y_test, y_pred)
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred, average='weighted')
print("Accuracy: {:.4f}, Precision: {:.4f}, Recall: {:.4f}, F1-score: {:.4f}".format(accuracy, precision, recall, f1_score))

In [ ]:
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
# Macro metrics
precision_macro = precision_score(y_test, y_pred, average='macro')
recall_macro = recall_score(y_test, y_pred, average='macro')
f1_macro = f1_score(y_test, y_pred, average='macro')

# Micro metrics
precision_micro = precision_score(y_test, y_pred, average='micro')
recall_micro = recall_score(y_test, y_pred, average='micro')
f1_micro = f1_score(y_test, y_pred, average='micro')

# Print metrics
print(f'Accuracy: {accuracy}')
print('Macro Metrics:')
print(f'Precision: {precision_macro}')
print(f'Recall: {recall_macro}')
print(f'F1-Score: {f1_macro}')

print('Micro Metrics:')
print(f'Precision: {precision_micro}')
print(f'Recall: {recall_micro}')
print(f'F1-Score: {f1_micro}')

In [ ]:
# Assuming y_test contains the true labels and y_pred contains the predicted labels

# Calculate classification report
target_names = ['1', '2', '3', '4', '5']
report = classification_report(y_test, y_pred, target_names=target_names, output_dict=True)

# Calculate accuracy separately
accuracy = accuracy_score(y_test, y_pred)

# Print precision, recall, f1-score, and accuracy for each class
for label, metrics in report.items():
    if label != 'accuracy':
        class_accuracy = metrics["support"] * metrics["recall"]
        print(f'Class: {label}')
        print(f'Class Accuracy: {class_accuracy}')
        print(f'Precision: {metrics["precision"]}')
        print(f'Recall: {metrics["recall"]}')
        print(f'F1-score: {metrics["f1-score"]}')
        print(f'Support: {metrics["support"]}')


# Print overall accuracy
print(f'Accuracy: {accuracy}')